In [1]:
# Awesome Text Only News
# https://github.com/localjo/awesome-text-only-news?tab=readme-ov-file

import requests
from bs4 import BeautifulSoup
import pandas as pd
# import sleep
import time

pd.set_option('display.max_colwidth', None)  # This ensures full text is shown

In [2]:
# Step 1: Fetch the webpage
base_url = "https://lite.cnn.com/"
response = requests.get(base_url)

print(response.status_code)

200


In [3]:
# Step 2: Parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
def get_article_content(link):
    try:
        article_response = requests.get(link)
        article_soup = BeautifulSoup(article_response.text, 'html.parser')
        
        # This assumes the main article content is in <p> tags, adjust if necessary
        paragraphs = article_soup.find_all('p')
        content = ' '.join([p.get_text() for p in paragraphs])
        return content.strip()
    except Exception as e:
        return f"Error fetching content: {e}"

In [5]:
# Step 3: Extract headlines and links
data = []
for headline in soup.find_all('a'):
    title = headline.get_text().strip()
    link = headline['href']
    # Construct full URL
    
    # Check if link is a relative URL, if so, prepend the base URL
    if link.startswith('/'):
        full_link = base_url + link
    elif link.startswith('http'):  # It's already a full link
        full_link = link
    else:
        # Handle unexpected cases
        full_link = base_url + '/' + link

    # Fetch the article content
    content = get_article_content(full_link)
    
    # Append title, link, and content to the data list
    data.append({"Title": title, "Link": full_link, "Content": content})
    time.sleep(0.2)  # Be nice to the server

# Convert to DataFrame
df = pd.DataFrame(data)

In [6]:
# Step 4: Print first 5 article titles and links for debugging
df.head(5)

Title  \
0                                                                                                             CNN   
1  African health officials hopeful mpox shots will start within days as WHO says outbreak is ‘not the new Covid’   
2                                            Oil giant Halliburton hit by apparent cyberattack disrupting systems   
3                                  ‘Seismic shift.’ Kroger’s fight against a federal agency could upend antitrust   
4                                         Biden spoke with Netanyahu Wednesday at a critical moment in Gaza talks   

                                                                                              Link  \
0                                                                           https://lite.cnn.com//   
1    https://lite.cnn.com//2024/08/20/africa/argentina-quarantines-mpox-ship-intl-latam/index.html   
2  https://lite.cnn.com//2024/08/21/business/oil-giant-halliburton-apparent-cyberattack/index.html   
3               https://lite.cnn.com//2024/08/20/business/kroger-albertsons-ftc-lawsuit/index.html   
4         https://lite.cnn.com//2024/08/21/politics/biden-netanyahu-call-gaza-ceasefire/index.html   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [7]:
OLLAMA_API_URL = "http://localhost:11434/api/generate"

In [9]:
import requests
import json

# Ollama API URL (default server address)
OLLAMA_API_URL = "http://localhost:11434/api/generate"

# Define the function to use Gemma 2 model in Ollama
def generate_response(prompt):
    # Define the payload with the prompt and model name
    payload = {
        "model": "gemma2:2b",  # Specify the model
        "prompt": prompt
    }
    
    try:
        # Send a request to Ollama API with streaming enabled
        response = requests.post(OLLAMA_API_URL, json=payload, stream=True)

        # Initialize an empty string to collect the full response
        full_response = ""

        # Iterate over the streaming chunks of the response
        for chunk in response.iter_lines():
            if chunk:
                # Convert the byte chunk to a string
                chunk_data = json.loads(chunk.decode("utf-8"))
                
                # Accumulate the "response" field
                full_response += chunk_data.get("response", "")
        
        # Return the full response generated by the model
        return full_response
    
    except requests.exceptions.RequestException as e:
        # Handle any exceptions during the request
        return f"Request failed: {e}"
    except json.JSONDecodeError as e:
        # Handle JSON decoding errors
        return f"JSON decode error: {e}"

# Example usage of the function
if __name__ == "__main__":
    # Provide a prompt to the model
    prompt_text = "Explain the concept of machine learning."
    # Generate a response from the Gemma 2 model
    result = generate_response(prompt_text)
    print("Response from Gemma 2:")
    print(result)

Response from Gemma 2:
Imagine teaching a dog a trick. You show them what to do, reward them when they get it right, and correct them when they make mistakes. Over time, the dog learns the task through repetition and feedback.

Machine learning is kind of like that for computers.  Instead of humans giving direct instructions, machine learning uses algorithms (like those "tricks" for dogs) to train computers based on data. 

Here's a breakdown:

**What it is:**

* **Data-driven:** Machine learning relies heavily on massive datasets. The more data it has, the better its predictions become.
* **Algorithmic learning:**  Algorithms are like recipes that teach machines how to analyze and learn from the data. These algorithms use mathematical formulas to identify patterns and make connections in the data. 
* **Goal-oriented:** Machine learning aims to create models that can perform specific tasks or solve problems, such as:
    * Recognizing images
    * Predicting customer behavior
    * Ide

In [12]:
df.shape

(106, 3)

In [13]:
df_sample = df.sample(5)
df_sample

Title  \
101                                                                             Go to the full CNN experience   
39                                   New data shows US job growth has been far weaker than initially reported   
11                          A tornado sank a luxury sailing boat off the coast of Sicily. Here’s what we know   
67   Prosecutors say this doctor profited off Matthew Perry’s addiction. He’s returning to practice this week   
43                                   See where abortions are banned and legal — and where it’s still in limbo   

                                                                                                Link  \
101                                                                             https://www.cnn.com/   
39                            https://lite.cnn.com//2024/08/21/economy/bls-jobs-revisions/index.html   
11               https://lite.cnn.com//2024/08/20/europe/bayesian-yacht-what-we-know-intl/index.html   
67   https://lite.cnn.com//2024/08/21/us/matthew-perry-salvador-plasencia-ketamine-doctor/index.html   
43                       https://lite.cnn.com//us/abortion-access-restrictions-bans-us-dg/index.html   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [17]:
# use generate_response function to generate summaries for each article, prompt cotain instructions for the model and title/content contain the article text
df_sample['Summary'] = df_sample.apply(lambda x: generate_response(f"Write exact tilte text first, add newline, then summarize the content of article in short paragraph in less than 3 sentences preserving key indicators or numbers \n\n titled '{x['Title']}'\n\n{x['Content']}"), axis=1)
df_sample.head(5)

Title  \
101                                                                             Go to the full CNN experience   
39                                   New data shows US job growth has been far weaker than initially reported   
11                          A tornado sank a luxury sailing boat off the coast of Sicily. Here’s what we know   
67   Prosecutors say this doctor profited off Matthew Perry’s addiction. He’s returning to practice this week   
43                                   See where abortions are banned and legal — and where it’s still in limbo   

                                                                                                Link  \
101                                                                             https://www.cnn.com/   
39                            https://lite.cnn.com//2024/08/21/economy/bls-jobs-revisions/index.html   
11               https://lite.cnn.com//2024/08/20/europe/bayesian-yacht-what-we-know-intl/index.html   
67   https://lite.cnn.com//2024/08/21/us/matthew-perry-salvador-plasencia-ketamine-doctor/index.html   
43                       https://lite.cnn.com//us/abortion-access-restrictions-bans-us-dg/index.html   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [4]:
# https://ai.stanford.edu/~amaas/data/sentiment/
# read positive reviews from aclImdb folder, train folder, pos folder, list all txt files and sampling 100 files
import os
import random

# Define the path to the positive reviews
pos_reviews_path = "aclImdb/train/pos"
neg_reviews_path = "aclImdb/train/neg"

# List all the files in the positive reviews folder
pos_files = os.listdir(pos_reviews_path)
neg_files = os.listdir(neg_reviews_path)
print(len(pos_files))
print(len(neg_files))

# Sample 100 files randomly
random.seed(0)
sampled_pos_files = random.sample(pos_files, 10)
sampled_neg_files = random.sample(neg_files, 10)
print(sampled_pos_files[:5])
print(sampled_neg_files[:5])

12500
12500
['8650_10.txt', '9723_9.txt', '6588_9.txt', '10388_7.txt', '2173_10.txt']
['11559_4.txt', '3299_1.txt', '2973_2.txt', '4415_4.txt', '11194_1.txt']


In [7]:
import pandas as pd
import os

pd.set_option('display.max_colwidth', None)  # None means no limit on column width

# set dataframe id, rating, review, label
# Create an empty list to store the data
data = []

# Iterate over the sampled positive files
for file in sampled_pos_files:
    # Read the content of the file
    with open(os.path.join(pos_reviews_path, file), "r", encoding="utf-8") as f:
        review = f.read()
    
    # Extract unique_id and rating from the file name
    unique_id, rating = file.split('_')  # Assuming file format is {unique_id}_{rating}.txt
    rating = rating.split('.')[0]  # Remove the .txt extension
    
    # Append the data in the format [id, rating, review, label]
    data.append([unique_id, rating, review, "positive"])  # Assuming the label is "positive"
   
for file in sampled_neg_files:
    # Read the content of the file
    with open(os.path.join(neg_reviews_path, file), "r", encoding="utf-8") as f:
        review = f.read()
    
    # Extract unique_id and rating from the file name
    unique_id, rating = file.split('_')  # Assuming file format is {unique_id}_{rating}.txt
    rating = rating.split('.')[0]  # Remove the .txt extension
    
    # Append the data in the format [id, rating, review, label]
    data.append([unique_id, rating, review, "negative"])  # Assuming the label is "positive"

# Create a DataFrame from the collected data
df_reviews = pd.DataFrame(data, columns=["id", "rating", "review", "label"])
df_reviews

,id,rating,review,label
0,8650,10,Kalifornia is the story of a writer and his girlfriend photographer who are looking for someone to help pay gas money and take turns at the wheel for a cross country road trip to famous murder sights. Ironically a serial killer and his girlfriend answer the post. Kalifornia is a diamond in the rough and a very intriguing journey with a serial killer. Great performances all around by the leads with Pitt in particular being exceptional. Check it out!!,positive
1,9723,9,"I really, really enjoyed watching this movie! At first, seeing its poster I thought it was just another easy romantic comedy ... but it is simply more than this! I personally believe that this idea (that I'm sure a good part of the viewers had just before they saw the movie) it's yet another important part of the big concept of this movie itself (or even of its marketing strategy)! What I mean is: Nowadays we are slaves to images! To impressions! I went to the cinema to view this film having the wrong impression, the wrong expectations, and at the end I felt how superficial I could be! To exemplify it comes to my mind the sequence near the end in which Sidney buys the plane ticket to go back to New York and as he is asked to 'give an autograph', meaning to sign for the ticket, he believes that just because he got on TV thanks to the scandal at the awards he is now some kind of celebrity. And this is just, I believe, the climax of this main theme around which the movies revolves. Above this, I believe the movie also offers us a solution to get along with this, illustrated throughout the movie by Sidney's attitude: don't become too serious about yourself or about anybody else ... ""even saints were people in the beginning"" ... as Sophie once says in the movie. The saints of the moment are the stars. We attribute them an 'aura' of perfection, of eternal happiness, but the reality is much less than that. Even the saints of any religion are images, ideal models of how to behave and how to live your life. Even they were not for real ... they became 'for real' after they died and we looked back at them. And that's the catch: we need our saints! we need our stars! We strive for them as if it wasn't for them we wouldn't have anything to strive for. And television and all other media are means to create and capture our strivings. We desperately need benchmarks in regard to which to measure ourselves. And that's how we got in the cinema to watch this movie in the first place: to see if we can fit the benchmark, or if the benchmark is to small for us. This time it was larger than we expected.",positive
2,6588,9,"After viewing several episodes of this series, I have come to the conclusion that television producers are completely devoid of any form of originality. Here is an old science fiction standby, ingeniously wrapped in the form of a truly original concept - and still they can only -almost - make it work.<br /><br />The dialog is good! The male actors are reasonably proficient at their professions. Most of the characters are well drawn, with special kudos to the hero and his more than likeable side-kick. And most of the episode plots come across as palatable. So what could be wrong? How about the, the female characters and the cosmeticly perfect actresses who are chosen to portray them. <br /><br />The producers insist on portraying the female characters in this - almost good - series, in a manner that makes the end product appear to be a misplaced cheerleader. Why, I ask, why?<br /><br />The episodes all fall flat whenever the female guest star or recurring character comes on screen. These actresses are all totally unbelievable in their roles, and you don't actually have to see them to know they are incapable of their acting assignments. A blind person could tell. Just listen to them talk. They deliver their dialog with all the drama and effect of a 16 year old at the high school prom. Who would believe these women are Phd scientist, s

In [13]:
# input review text to generate sentiment analysis using generate_response function
df_reviews['Sentiment'] = df_reviews.apply(lambda x: generate_response(f"Analyze the sentiment of the following review and return only positive or negative string: \n\n {x['review']}"), axis=1)

In [14]:
df_reviews

,id,rating,review,label,Sentiment
0,8650,10,Kalifornia is the story of a writer and his girlfriend photographer who are looking for someone to help pay gas money and take turns at the wheel for a cross country road trip to famous murder sights. Ironically a serial killer and his girlfriend answer the post. Kalifornia is a diamond in the rough and a very intriguing journey with a serial killer. Great performances all around by the leads with Pitt in particular being exceptional. Check it out!!,positive,positive \n
1,9723,9,"I really, really enjoyed watching this movie! At first, seeing its poster I thought it was just another easy romantic comedy ... but it is simply more than this! I personally believe that this idea (that I'm sure a good part of the viewers had just before they saw the movie) it's yet another important part of the big concept of this movie itself (or even of its marketing strategy)! What I mean is: Nowadays we are slaves to images! To impressions! I went to the cinema to view this film having the wrong impression, the wrong expectations, and at the end I felt how superficial I could be! To exemplify it comes to my mind the sequence near the end in which Sidney buys the plane ticket to go back to New York and as he is asked to 'give an autograph', meaning to sign for the ticket, he believes that just because he got on TV thanks to the scandal at the awards he is now some kind of celebrity. And this is just, I believe, the climax of this main theme around which the movies revolves. Above this, I believe the movie also offers us a solution to get along with this, illustrated throughout the movie by Sidney's attitude: don't become too serious about yourself or about anybody else ... ""even saints were people in the beginning"" ... as Sophie once says in the movie. The saints of the moment are the stars. We attribute them an 'aura' of perfection, of eternal happiness, but the reality is much less than that. Even the saints of any religion are images, ideal models of how to behave and how to live your life. Even they were not for real ... they became 'for real' after they died and we looked back at them. And that's the catch: we need our saints! we need our stars! We strive for them as if it wasn't for them we wouldn't have anything to strive for. And television and all other media are means to create and capture our strivings. We desperately need benchmarks in regard to which to measure ourselves. And that's how we got in the cinema to watch this movie in the first place: to see if we can fit the benchmark, or if the benchmark is to small for us. This time it was larger than we expected.",positive,positive \n
2,6588,9,"After viewing several episodes of this series, I have come to the conclusion that television producers are completely devoid of any form of originality. Here is an old science fiction standby, ingeniously wrapped in the form of a truly original concept - and still they can only -almost - make it work.<br /><br />The dialog is good! The male actors are reasonably proficient at their professions. Most of the characters are well drawn, with special kudos to the hero and his more than likeable side-kick. And most of the episode plots come across as palatable. So what could be wrong? How about the, the female characters and the cosmeticly perfect actresses who are chosen to portray them. <br /><br />The producers insist on portraying the female characters in this - almost good - series, in a manner that makes the end product appear to be a misplaced cheerleader. Why, I ask, why?<br /><br />The episodes all fall flat whenever the female guest star or recurring character comes on screen. These actresses are all totally unbelievable in their roles, and you don't actually have to see them to know they are incapable of their acting assignments. A blind person could tell. Just listen to them talk. They deliver their dialog with all the drama and effect of a 16 year old at the high school prom. Who would believ

In [15]:
from crawl4ai import WebCrawler
crawler = WebCrawler()
crawler.warmup()
result = crawler.run(url="https://cdn.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN=")
print(result.markdown)

[LOG] 🚀 Initializing LocalSeleniumCrawlerStrategy
[LOG] 🌤️  Warming up the WebCrawler
[LOG] 🌞 WebCrawler is ready to crawl
[LOG] 🚀 Crawling done for https://cdn.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN=, success: True, time taken: 0.4092729091644287 seconds
[LOG] 🚀 Content extracted for https://cdn.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN=, success: True, time taken: 0.1468198299407959 seconds
[LOG] 🔥 Extracting semantic blocks for https://cdn.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN=, Strategy: NoExtractionStrategy
[LOG] 🚀 Extraction done for https://cdn.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN=, time taken: 0.14783191680908203 seconds.
본문 바로가기 메뉴 바로가기

# 상장기업분석

리포트 조회 통합 모바일앱 리타민 출시리포트 조회 통합 모바일앱 리타민

In [ ]:
from crawl4ai import WebCrawler
from crawl4ai.extraction_strategy import LLMExtractionStrategy
from pydantic import BaseModel, Field

def get_crawler():
    crawler = WebCrawler(verbose=True)
    crawler.warmup()
    return crawler

class PageSummary(BaseModel):
    title: str = Field(..., description="Title of the page.")
    summary: str = Field(..., description="Summary of the page.")
    brief_summary: str = Field(..., description="Brief summary of the page.")
    keywords: list = Field(..., description="Keywords assigned to the page.")

def summary_extraction(url, model_name='openai/gpt-4o'):
    result = get_crawler().run(
        url=url,
        word_count_threshold=1,
        extraction_strategy=LLMExtractionStrategy(
            provider= model_name, 
            api_token = os.getenv('OPENAI_API_KEY'), 
            schema=PageSummary.model_json_schema(),
            extraction_type="schema",
            apply_chunking=False,
            instruction=(
                "From the crawled content, extract the following details: "
                "1. Title of the page "
                "2. Summary of the page, which is a detailed summary "
                "3. Brief summary of the page, which is a paragraph text "
                "4. Keywords assigned to the page, which is a list of keywords. "
                'The extracted JSON format should look like this: '
                '{ "title": "Page Title", "summary": "Detailed summary of the page.", '
                '"brief_summary": "Brief summary in a paragraph.", "keywords": ["keyword1", "keyword2", "keyword3"] }'
            )
        ),
        bypass_cache=True,
    )
    result = result.extracted_content.encode('utf-8', errors='ignore').decode("unicode_escape")
    result_json = json.loads(result)
    return result_json